# U-Net Depth (Projeto de dissertação)

In [ ]:
running_on_kaggle = False
running_on_colab = False
running_remote = running_on_kaggle or running_on_colab
repository_name = 'unet_depth'

## Clonar e navegar para o repositório

In [ ]:
if running_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import os

# Clone e navegue para o repositório da dissertação
if running_remote:
    !git clone "https://github.com/duraes-antonio/unet_depth"
    !cd $repository_name
    os.chdir(repository_name)

In [ ]:
!git checkout "feat/automatizar_config_rede"

In [ ]:
from domain.models.network import NetworkConfig

config: NetworkConfig = {
    'size': 256,
    'filter_min': 64,
    'filter_max': 512,
    'pool': True,
    'unpool': True,
}
db_name = f"{config['size']}_{config['filter_min']}-{config['filter_max']}_pool-{config['pool']}_unpool-{config['unpool']}"

## Definir variáveis de ambiente e credenciais para serviços

In [ ]:
!pip install pymongo[srv] dnspython keras-unet-collection python-dotenv imutils
!pip install py-cpuinfo gpuinfo typing_extensions

if not running_remote:
    !pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import ast
import json
import dotenv

if running_on_kaggle:
    from kaggle_secrets import UserSecretsClient

    user_secrets = UserSecretsClient()
    database_url = user_secrets.get_secret("DATABASE_URL")
    google_cred_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_CREDENTIALS_JSON"))
    google_token_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_TOKEN_JSON"))
    os.environ["DATABASE_URL"] = database_url
    os.environ["DB_NAME"] = db_name

else:
    if running_on_colab:
        path_auth = '../drive/MyDrive/unet_depth/auth/.env'
    else:
        path_auth = './.env'

    dotenv.load_dotenv(path_auth)
    database_url = os.environ["DATABASE_URL"]
    google_cred_json = ast.literal_eval(os.environ["GOOGLE_CREDENTIALS_JSON"])
    google_token_json = ast.literal_eval(os.environ["GOOGLE_TOKEN_JSON"])

with open('google_credentials.json', 'w', encoding='utf-8') as f:
    json.dump(google_cred_json, f, ensure_ascii=False, indent=4)

with open('token.json', 'w', encoding='utf-8') as f:
    json.dump(google_token_json, f, ensure_ascii=False, indent=4)

## Instanciar serviços para persistência de resultados e blob

In [ ]:
from infra.services.results_service_mongodb import ResultServiceMongoDB
from domain.services.results_service import ResultService
from infra.services.blob_storage.blob_storage_service_google_drive import GoogleDriveBlobStorageService
from infra.services.model_storage_service_google_drive import ModelStorageServiceGoogleDrive
from domain.services.model_storage_service import ModelStorageService
from infra.services.test_case_execution_service_mongodb import TestCaseExecutionServiceMongoDB
from infra.services.test_case_service_mongodb import TestCaseServiceMongoDB
from domain.services.test_case_execution_service import TestCaseExecutionService
from domain.services.blob_storage_service import BlobStorageService
from domain.services.test_case_service import TestCaseService

test_case_serv: TestCaseService = TestCaseServiceMongoDB(db_name)
execution_serv: TestCaseExecutionService = TestCaseExecutionServiceMongoDB(db_name)

blob_service: BlobStorageService = GoogleDriveBlobStorageService(db_name)
model_storage: ModelStorageService = ModelStorageServiceGoogleDrive(blob_service)

result_service: ResultService = ResultServiceMongoDB(db_name)

## Baixar dataset

In [ ]:
if not os.path.exists('data'):
    !git clone "https://gitlab.com/siddinc/new_depth.git" "./data"

## Executar aplicação

In [ ]:
from tensorflow import keras
from infra.util.dataset import read_nyu_csv
from infra.util.output import plot_image_comparison
from infra.application_manager import ApplicationManager

train_path = "./data/nyu2_train.csv"
test_path = "./data/nyu2_test.csv"
batch_size = 4
epochs = 70

# Defina a semente usada em operações como embaralhamento do dataset
seed = 42


# callback após avaliação em base de teste
def after_test(model: keras.Model):
    xy_path_pairs = read_nyu_csv(test_path)
    return plot_image_comparison(model, xy_path_pairs)


try:
    application = ApplicationManager(
        blob_service, model_storage, execution_serv,
        test_case_serv, result_service, config, epochs
    )
    application.prepare_train_data(train_path, batch_size, True, seed, 1)
    application.prepare_test_data(test_path, batch_size, True, seed)
    application.run()

except Exception as E:
    print(E)